In [46]:
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir("C:\\Users\\ggorski\\OneDrive - DOI\\USGS_ML\\Estuary Salinity\\github\\drb-estuary-salinity-ml")
import pandas as pd
import pickle
from LSTMDA_torch import LSTMDA, fit_torch_model, rmse_masked
import seaborn as sns
import sys
sys.path.append('03b_model/src')
import torch
import run_model
import yaml


In [49]:
#read prepped data in
prepped_model_io_data_file = '04_model_analysis/out/prepped_model_io_data'
with open(prepped_model_io_data_file, 'rb') as f:
    prepped_model_io_data = pickle.load(f)

In [51]:
#set parameters using config file
with open("04_model_analysis/model_config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

#x variables
inputs = config['inputs']
#model target
target = config['target']

#other params from config file
seq_len = config['seq_len']
offset = config['offset']
n_epochs = config['n_epochs']
hidden_units = config['hidden_units']
learn_rate = config['learn_rate']
recur_dropout = config['recur_dropout']
dropout = config['dropout']

n_batch, seq_len, n_feat  = prepped_model_io_data['trainval_features'].shape

In [52]:
prepped_model_io_data['train_features'].shape


torch.Size([20, 180, 10])

In [53]:
pretrain_model = LSTMDA(n_feat, hidden_units, recur_dropout, dropout)

pretrain_model.load_state_dict(torch.load('04_model_analysis/out/Run_XAI_test_01/weights.pt')) # ensure that dropout layers are active

y_pred, _ = pretrain_model(prepped_model_io_data['trainval_features'])

In [54]:
pretrain_model

LSTMDA(
  (dropout): Dropout(p=0.1, inplace=False)
  (recur_dropout): Dropout(p=0.0, inplace=False)
  (dense): Linear(in_features=10, out_features=1, bias=True)
)